In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold,GroupKFold
import math
import scipy
import numpy as np
import pandas as pd
import sklearn.metrics as mtr 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout, PReLU, BatchNormalization, ELU, GaussianNoise, Activation
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
from sklearn import preprocessing
import gc
import os
from tqdm import tqdm_notebook
import keras.backend as K
#note： 
#1. As a result it might not be worthwhile to use features related to game clock/quarter of the game。
#2. There is no relationships between number of rushes before and running yards gained。
#3. rushing success larger depends on defender in box, or defender that are close to offensive lineman and attempt 
#   to counter the blocking.
#4. highly drafted player has the same average rushing yards as the rest

In [46]:
train_df = pd.read_csv(r'C:\Users\38980\OneDrive\Desktop\study\kaggle\NFL\train.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [214]:
#team abbreviations correct
def data_clean(df):
#correct name   
    df.loc[df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
    df.loc[df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
    df.loc[df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
    df.loc[df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
    df.loc[df['FieldPosition'] == 'ARZ', 'FieldPosition'] = 'ARI'
    df.loc[df['FieldPosition'] == 'BLT', 'FieldPosition'] = 'BAL'
    df.loc[df['FieldPosition'] == 'CLV', 'FieldPosition'] = 'CLE'
    df.loc[df['FieldPosition'] == 'HST', 'FieldPosition'] = 'HOU'

# fill null
    df = df.fillna(df.median())

# offense time and defence time
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    
#time
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df['GameClock'] = df['GameClock'].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
#player height
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

#weather
    def map_weather(txt):
        ans = 1
        if pd.isna(txt):
            return 0
        if 'partly' in txt:
            ans*=0.5
        if 'climate controlled' in txt or 'indoor' in txt:
            return ans*3
        if 'sunny' in txt or 'sun' in txt:
            return ans*2
        if 'clear' in txt:
            return ans
        if 'cloudy' in txt:
            return -ans
        if 'rain' in txt or 'rainy' in txt:
            return -2*ans
        if 'snow' in txt:
            return -3*ans
        return 0
    df['Cleaned_GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['Cleaned_GameWeather'].apply(map_weather)

#diff Score    
    df["DiffScoreBeforePlay_ob"] = (df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"])
    df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]] = - df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]]
#Turf
    def agrupar_gramado(Turf):
        if Turf == 'Artifical':
            return 'Artificial'

        elif Turf in ('FieldTurf', 'Field turf'):
            return 'Field Turf'

        elif Turf in ('FieldTurf360', 'FieldTurf 360'):
            return 'Field Turf 360'

        elif Turf in ('Natural', 'Natural grass', 'Naturall Grass', 'grass', 'natural grass', 'SISGrass', 'Natural Grass'):
            return "Grass"

        elif Turf == "UBU Sports Speed S5-M":
            return "UBU Speed Series-S5-M"

        else:
            return Turf
        df['Turf'] = df['Turf'].apply(agrupar_gramado)

#left to right
    df['New_X'] = df['X']
    df.loc[df['PlayDirection'] == 'left','New_X'] = 120 - df.loc[df['PlayDirection'] == 'left','X']
    df['New_Y'] = df['Y']
    df.loc[df['PlayDirection'] == 'left','New_Y'] = 160/3 - df.loc[df['PlayDirection'] == 'left','Y']
    df['Orientation_std'] = df['Orientation']
    df.loc[df['Season'] == 2017, 'Orientation_std'] = df.loc[df['Season'] == 2017, 'Orientation_std'] + 90
    #df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std'] = 360 - df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std']
    df.loc[df['PlayDirection'] == 'left', 'Orientation_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation_std'], 360)
    df['Dir_std'] = df['Dir']
    #df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = 360 - df.loc[df['PlayDirection'] == 'left', 'Dir_std']
    df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir_std'], 360)
    df['YardLine_std'] = 100 - df['YardLine']
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine_std'
         ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine']
    df["Orientation_sin"] = df["Orientation_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Orientation_cos"] = df["Orientation_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    df["Dir_sin"] = df["Dir_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Dir_cos"] = df["Dir_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

#distance and S
    #distance to yardline
    df['Dis_YardLine'] = df['New_X'] - df['YardLine_std'] - 10
    #distance to rusher
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    Rusher =df.loc[df['IsRusher'],['PlayId','X','Y','Dir_std','S']].rename(columns={"X":"Rusher_X","Y":"Rusher_Y",'PossessionTeam':'Offense_Team','Dir_std':'Rusher_Dir_std','S':'Rusher_Speed'})
    df = df.merge(Rusher,how = 'left',on = 'PlayId')
    df['Distance_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_Diff'] = df['Degree_to_Rusher'] - df['Rusher_Dir_std']
    df['Speed_Ratio'] =  df['Rusher_Speed']/df['S']
    #back_direction
    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0
    df['back_oriented_down_field'] = df['Orientation_std'].apply(lambda x: back_direction(x))

# Stadium and location clean
    def agrupar_local(Location):
        if Location == "Arlington, Texas":
            return "Arlington, TX"
        elif Location in ("Baltimore, Maryland","Baltimore, Md."):
            return "Baltimore, MD"
        elif Location == "Charlotte, North Carolina":
            return "Charlotte, NC"
        elif Location == "Chicago. IL":
            return "Chicago, IL"
        elif Location == "Cincinnati, Ohio":
            return "Cincinnati, OH"
        elif Location in ("Cleveland","Cleveland Ohio","Cleveland, Ohio","Cleveland,Ohio"):
            return "Cleveland, OH"
        elif Location == "Detroit":
            return "Detroit, MI"
        elif Location == "E. Rutherford, NJ" or Location == "East Rutherford, N.J.":
            return "East Rutherford, NJ"
        elif Location == "Foxborough, Ma":
            return "Foxborough, MA"
        elif Location == "Houston, Texas":
            return "Houston, TX"
        elif Location in ("Jacksonville Florida","Jacksonville, Fl","Jacksonville, Florida"):
            return "Jacksonville, FL"
        elif Location == "London":
            return "London, England"
        elif Location == "Los Angeles, Calif.":
            return "Los Angeles, CA"
        elif Location == "Miami Gardens, Fla.":
            return "Miami Gardens, FLA"
        elif Location in ("New Orleans","New Orleans, La."):
            return "New Orleans, LA"
        elif Location == "Orchard Park NY":
            return "Orchard Park, NY"
        elif Location == "Philadelphia, Pa.":
            return "Philadelphia, PA"
        elif Location == "Pittsburgh":
            return "Pittsburgh, PA"
        elif Location == "Seattle":
            return "Seattle, WA"
        else:
            return Location

    df['Location'] = df['Location'].apply(agrupar_local)

# stadium types
    def agrupar_tipo_estadio(StadiumType):
        outdoor       = ['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
        indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
        indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
        dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
        dome_open     = ['Domed, Open', 'Domed, open']

        if StadiumType in outdoor:
            return 'outdoor'
        elif StadiumType in indoor_closed:
            return 'indoor_closed'
        elif StadiumType in indoor_open:
            return 'indoor_open'
        elif StadiumType in dome_closed:
            return 'dome_closed'
        elif StadiumType in dome_open:
            return 'dome_open'
        else:
            return 'unknown'
    df['StadiumType'] = df['StadiumType'].apply(agrupar_tipo_estadio)

# wind 
    def give_me_WindSpeed(x):
            x = str(x)
            x = x.replace('mph', '').strip()
            if '-' in x:
                x = (int(x.split('-')[0]) + int(x.split('-')[1])) / 2
            elif 'gusts up to' in x:
                x = (int(x.split()[0]) + int(x.split()[-1])) / 2
            elif 'clam' in x:
                x = 0
            try:
                return float(x)
            except:
                return -99
    df['Cleaned_WindSpeed'] = df['WindSpeed'].apply(give_me_WindSpeed)

# wind direction
    def agrupa_wind_direction(WindDirection):
        wd = str(WindDirection).upper()

        if wd == 'N' or 'FROM N' in wd:
            return 'north'
        if wd == 'S' or 'FROM S' in wd:
            return 'south'
        if wd == 'W' or 'FROM W' in wd:
            return 'west'
        if wd == 'E' or 'FROM E' in wd:
            return 'east'

        if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
            return 'south west'
        if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
            return 'south east'
        if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
            return 'north west'
        if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
            return 'north east'

        if 'NW' in wd or 'NORTHWEST' in wd:
            return 'north west'
        if 'NE' in wd or 'NORTH EAST' in wd:
            return 'north east'
        if 'SW' in wd or 'SOUTHWEST' in wd:
            return 'south west'
        if 'SE' in wd or 'SOUTHEAST' in wd:
            return 'south east'

        return 'unknown'
    
    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)

# speed
    df.loc[df['Season'] == 2017, 'S'] = (df.loc[df['Season'] == 2017, 'S'] - 2.4355) / 1.2930 * 1.4551 + 2.7570
    df['Horizontal Speed'] = df['S']*df["Dir_sin"]
    df['Vertical Speed'] = df['S']*df["Dir_cos"]
# momentum
    df['momentum'] = df['S']*df['PlayerWeight']

    
    return df

In [215]:
train = data_clean(train_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in double_scalars


In [216]:
def split_data(df):
    df1 = df.loc[df['IsRusher']]
    df2 = df.loc[df['IsOnOffense'] & (~df['IsRusher'])]
    df3 = df.loc[~df['IsOnOffense']]

# max yards
    df1['Max_Yards'] = df1['YardLine']
    df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards']
    df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards']
# min_time_to_tacke
    df3['Min_Time_Tacke'] = df3['Distance_to_Rusher']/df3['S']
    df3.loc[df3['Min_Time_Tacke'] == np.inf, 'Min_Time_Tacke'] = 20
# defence_X_Y_spread
    Defence_X_Y_std = df3[["PlayId",'New_X','New_Y']].groupby("PlayId").std().rename(columns={'New_X':'Defense_X_std','New_Y':'Defense_Y_std'}) \
    .reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed2_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed2_std'}).reset_index()

    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed2_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed2_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed4_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed4_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed4_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Defence_X_Y_std, how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed4_std,how = 'left',  on ='PlayId')

#distance to QB
    dis_QB = df2.loc[df2["Position"] =='QB',['PlayId','Distance_to_Rusher']].groupby(['PlayId']).mean().rename(columns={'Distance_to_Rusher':'dis_to_QB'})
    df1 = df1.merge(dis_QB,how = 'left', on='PlayId')
#defence min,max,mean,std distance to rusher
    stat = df3.groupby(['GameId','PlayId']).agg({'Distance_to_Rusher':['min','max','mean','std']})
    stat.columns = stat.columns.droplevel()
    df1 = df1.merge(stat,how = 'left', on='PlayId')

# offense_X_Y_spread
    df2 = df2.sort_values(['PlayId','New_X'])
    Offense_X_Removed2_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed2_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed2_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed2_std'}).reset_index()
    
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_X_Removed4_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed4_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed4_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Offense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed4_std,how = 'left',  on ='PlayId')
    
# nearest offenders to defender
    dis_to_closest_offender = pd.DataFrame()
    for playid in df3['PlayId'].unique():
        offense = df2.loc[df2['PlayId'] == playid]
        defence = df3.loc[df3['PlayId'] == playid]
        ary = scipy.spatial.distance.cdist(defence[['New_X','New_Y']], offense[['New_X','New_Y']], metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, defence[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_offender = dis_to_closest_offender.append(ary)
    df3 = df3.merge(dis_to_closest_offender,how = 'left',on = ['PlayId','NflId'])

# personnel_features
    def defense_formation(l):
        dl = 0
        lb = 0
        db = 0
        other = 0

        for position in l:
            sub_string = position.split(' ')
            if sub_string[1] == 'DL':
                dl += int(sub_string[0])
            elif sub_string[1] in ['LB','OL']:
                lb += int(sub_string[0])
            else:
                db += int(sub_string[0])

        counts = (dl,lb,db,other)

        return counts
    def offense_formation(l):
        qb = 0
        rb = 0
        wr = 0
        te = 0
        ol = 0

        sub_total = 0
        qb_listed = False
        for position in l:
            sub_string = position.split(' ')
            pos = sub_string[1]
            cnt = int(sub_string[0])

            if pos == 'QB':
                qb += cnt
                sub_total += cnt
                qb_listed = True
            # Assuming LB is a line backer lined up as full back
            elif pos in ['RB','LB']:
                rb += cnt
                sub_total += cnt
            # Assuming DB is a defensive back and lined up as WR
            elif pos in ['WR','DB']:
                wr += cnt
                sub_total += cnt
            elif pos == 'TE':
                te += cnt
                sub_total += cnt
            # Assuming DL is a defensive lineman lined up as an additional line man
            else:
                ol += cnt
                sub_total += cnt

        # If not all 11 players were noted at given positions we need to make some assumptions
        # I will assume if a QB is not listed then there was 1 QB on the play
        # If a QB is listed then I'm going to assume the rest of the positions are at OL
        # This might be flawed but it looks like RB, TE and WR are always listed in the personnel
        if sub_total < 11:
            diff = 11 - sub_total
            if not qb_listed:
                qb += 1
                diff -= 1
            ol += diff

        counts = (qb,rb,wr,te,ol)

        return counts
    def split_personnel(s):
        splits = s.split(',')
        for i in range(len(splits)):
            splits[i] = splits[i].strip()

        return splits    
    def personnel_features(df):
        personnel = df[['GameId','PlayId','OffensePersonnel','DefensePersonnel']].drop_duplicates()
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: defense_formation(x))
        personnel['num_DL'] = personnel['DefensePersonnel'].apply(lambda x: x[0])
        personnel['num_LB'] = personnel['DefensePersonnel'].apply(lambda x: x[1])
        personnel['num_DB'] = personnel['DefensePersonnel'].apply(lambda x: x[2])

        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: offense_formation(x))
        personnel['num_QB'] = personnel['OffensePersonnel'].apply(lambda x: x[0])
        personnel['num_RB'] = personnel['OffensePersonnel'].apply(lambda x: x[1])
        personnel['num_WR'] = personnel['OffensePersonnel'].apply(lambda x: x[2])
        personnel['num_TE'] = personnel['OffensePersonnel'].apply(lambda x: x[3])
        personnel['num_OL'] = personnel['OffensePersonnel'].apply(lambda x: x[4])

        # Let's create some features to specify if the OL is covered
        personnel['OL_diff'] = personnel['num_OL'] - personnel['num_DL']
        personnel['OL_TE_diff'] = (personnel['num_OL'] + personnel['num_TE']) - personnel['num_DL']
        # Let's create a feature to specify if the defense is preventing the run
        # Let's just assume 7 or more DL and LB is run prevention
        personnel['run_def'] = (personnel['num_DL'] + personnel['num_LB'] > 6).astype(int)

        personnel.drop(['OffensePersonnel','DefensePersonnel'], axis=1, inplace=True)
        
        return personnel
    
    personnel = personnel_features(df1)   
    df1 = df1.merge(personnel,how = 'left',  on ='PlayId')
    
#select useful columns    
    rusher = df1[['PlayId','TimeDelta','Team','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y', \
                 'Orientation_std','Dir_std','Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position',\
                 'Quarter','GameClock','Down','Distance','OffenseFormation',\
                  'DefendersInTheBox','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                 'Offense_X_Removed2_std','Offense_Y_Removed2_std','Offense_X_Removed4_std','Offense_Y_Removed4_std',\
                 'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std','Offense_Y_Removed2_std','Defense_X_Removed4_std',\
                 'Defense_Y_Removed4_std',\
                  'num_DL','num_LB','num_DB','num_QB','num_RB','num_WR','num_TE','num_OL','OL_diff','OL_TE_diff','run_def',\
                 'min','max','std','mean','dis_to_QB','Max_Yards']]
    rusher = rusher.sort_values('PlayId')
    game = df1[['PlayId','Cleaned_GameWeather','Humidity','Temperature', \
              'Week','WindDirection','Cleaned_WindSpeed','Location','StadiumType','PlayDirection']]
    game = game.sort_values('PlayId')
    offender = df2[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff']]
    offender = offender.sort_values(['PlayId','Distance_to_Rusher'])
    defender = df3[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff','Min_Time_Tacke','Speed_Ratio']+list(range(2))] #list(range(2)) two closest offenders
    defender = defender.sort_values(['PlayId','Distance_to_Rusher'])

    
    return rusher, game, offender, defender

In [217]:
rusher, game, offender, defender = split_data(train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

In [218]:
#remove some columns
rusher = rusher.drop('max',axis = 1)
game = game.drop(['Humidity'], axis = 1)

offender = offender.drop(['back_oriented_down_field'],axis = 1)
offender = offender.sort_values(['PlayId','Distance_to_Rusher']).groupby('PlayId').head(2)
defender = defender.sort_values(['PlayId','Distance_to_Rusher']).drop(['New_Y','back_oriented_down_field','Position'],axis = 1).groupby('PlayId').head(5)


In [219]:
#encoding fit
le = preprocessing.LabelEncoder()
categories =[]
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = rusher[i].astype(str)
    categories.append(rusher[i].unique())

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = game[i].astype(str)
    categories.append(game[i].unique())

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = offender[i].astype(str)
    categories.append(offender[i].unique())

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = defender[i].astype(str)
    categories.append(defender[i].unique())
categories = np.hstack(categories)
le.fit(categories)
le.classes_

array(['ACE', 'Arlington, TX', 'Atlanta, GA', 'Baltimore, MD', 'C', 'CB',
       'Carson, CA', 'Charlotte, NC', 'Chicago, IL', 'Cincinnati, OH',
       'Cleveland, OH', 'DE', 'DT', 'Denver, CO', 'Detroit, MI', 'EMPTY',
       'East Rutherford, NJ', 'FB', 'Foxborough, MA', 'G', 'Glendale, AZ',
       'Green Bay, WI', 'HB', 'Houston, TX', 'I_FORM',
       'Indianapolis, Ind.', 'JUMBO', 'Jacksonville, FL',
       'Kansas City,  MO', 'Kansas City, MO', 'Landover, MD',
       'London, England', 'Los Angeles, CA', 'Mexico City',
       'Miami Gardens, FLA', 'Minneapolis, MN', 'NT', 'Nashville, TN',
       'New Orleans, LA', 'OG', 'OLB', 'OT', 'Oakland, CA',
       'Orchard Park, NY', 'PISTOL', 'Philadelphia, PA', 'Pittsburgh, PA',
       'QB', 'RB', 'SHOTGUN', 'SINGLEBACK', 'Santa Clara, CA',
       'Seattle, WA', 'T', 'TE', 'Tampa, FL', 'WILDCAT', 'WR', 'away',
       'dome_closed', 'dome_open', 'east', 'home', 'indoor_closed',
       'indoor_open', 'left', 'nan', 'north', 'north east', 'no

In [220]:
#encoding transform
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = le.transform(rusher[i])

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = le.transform(game[i])

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = le.transform(offender[i])

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = le.transform(defender[i])

In [221]:
#reshape offender
offender_players = [offender.drop('PlayId',axis = 1).iloc[np.arange(k, len(offender), 2)].reset_index(drop = True) for k in range(2)]
offender_players = np.hstack([t.values for t in offender_players])
offender_players.shape

(23171, 32)

In [222]:
#reshape defender
defender_players = [defender.drop('PlayId',axis = 1).iloc[np.arange(k, len(defender), 5)].reset_index(drop = True) for k in range(5)]
defender_players = np.hstack([t.values for t in defender_players])
defender_players.shape

(23171, 90)

In [223]:
#train
train_y =train.loc[train['IsRusher'],['Yards','PlayId']].sort_values('PlayId').drop('PlayId',axis = 1)
train_x = np.hstack([rusher.drop('PlayId',axis = 1).values,defender_players,offender_players])
train_y99 =(train_y + 99).reset_index(drop=True).Yards

In [224]:
#loss function
def crps_eval(y_pred, dataset, is_higher_better=False):
    labels = dataset.get_label()
    labels = labels.astype('int')
    y_true = np.zeros((len(labels),199))
    for i, v in enumerate(labels):
        y_true[i, v:] = 1
    y_pred = y_pred.reshape(-1, 199, order='F')
    y_pred = np.clip(y_pred.cumsum(axis=1), 0, 1)
    return 'crps', np.mean((y_pred - y_true)**2), False

In [225]:
# hyper parameters
params = {'max_depth':5, 'lambda_l1': 1.2, 'lambda_l2': 1.2,
 'num_leaves': 32, 'feature_fraction': 0.4,
 'subsample': 0.4, 'min_child_samples': 15,
 'learning_rate': 0.02,
 'num_iterations': 1000, 'random_state': 42,
 'objective': 'multiclass',
 'min_gain_to_split':0.9,
 'num_class':199,
 'metric':'None'}


In [226]:
import lightgbm as lgb
for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              valid_sets=[dtrain,dvalid],
                              early_stopping_rounds=7,
                              verbose_eval=100,
                              feval=crps_eval)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 7 rounds
[100]	training's crps: 0.0126837	valid_1's crps: 0.0133622
[200]	training's crps: 0.0115451	valid_1's crps: 0.0130624
[300]	training's crps: 0.0108678	valid_1's crps: 0.012931
[400]	training's crps: 0.0104331	valid_1's crps: 0.0128705
[500]	training's crps: 0.0101361	valid_1's crps: 0.012836
[600]	training's crps: 0.00990675	valid_1's crps: 0.0128169
[700]	training's crps: 0.00974295	valid_1's crps: 0.0128032
[800]	training's crps: 0.00962582	valid_1's crps: 0.0127954
[900]	training's crps: 0.00955819	valid_1's crps: 0.0127895
Early stopping, best iteration is:
[992]	training's crps: 0.00952965	valid_1's crps: 0.0127868
Training until validation scores don't improve for 7 rounds
[100]	training's crps: 0.0127112	valid_1's crps: 0.013276
[200]	training's crps: 0.0115912	valid_1's crps: 0.012978
[300]	training's crps: 0.010922	valid_1's crps: 0.0128523
[400]	training's crps: 0.0105043	valid_1's crps: 0.0127904
[500]	training's cr

In [227]:
#CV score
# for leave out one feature importance compare
print(np.mean([0.0129045,0.0128677,0.0127771,0.0127333,0.013174]))

# 5 closest defense palyer drop 'New_Y','back_oriented_down_field',max
print(np.mean([0.0128208, 0.0127364, 0.0126515, 0.0126222,0.0130614]))

# all defense palyer
print(np.mean([0.012802,0.0127486,0.0126934,0.0126226,0.0130668]))

# 5 closest defender + 2 closest offender l1,l2 = 1.2
print(np.mean([0.0127868,0.0127041,0.0126144,0.0125952,0.0130483]))



0.012891320000000001
0.01277846
0.01278668
0.012749759999999999


In [ ]:
from sklearn.model_selection import ParameterGrid

params_dic = {'max_depth':[5], 'lambda_l1': [0.8,1,1.2], 'lambda_l2': [0.8,1,1.2],
 'num_leaves': [32], 'feature_fraction': [0.4,0.45],
 'subsample': [0.45], 'min_child_samples': [15],
 'learning_rate': [0.02],
 'num_iterations': [1000], 'random_state': [42],
 'objective': ['multiclass'],
 'min_gain_to_split':[1,1.2,1.4],
 'num_class':[199],
 'metric':['None']}

for params in list(ParameterGrid(params_dic)):
    kfold = KFold(5, random_state = 42, shuffle = True)
    print(params)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              valid_sets=[dtrain,dvalid],
                              early_stopping_rounds=7,
                              verbose_eval=1000,
                              feval=crps_eval)

In [66]:
#add one feature out test for game features
#[0.0128264, 0.0127599, 0.0126589, 0.0126459,0.0130807]
import lightgbm as lgb
train_y =train.loc[train['IsRusher'],['Yards','PlayId']].sort_values('PlayId').drop('PlayId',axis = 1)
train_y99 =(train_y + 99).reset_index(drop=True).Yards
offender_players = [offender.iloc[np.arange(k, len(offender), 1)].reset_index(drop = True) for k in range(1)]
offender_players = np.hstack([t.values for t in offender_players])
train_x = np.hstack([rusher.drop('PlayId',axis = 1).values,offender_players])

for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                                num_boost_round=100000,
                                valid_sets=[dtrain,dvalid],
                                early_stopping_rounds=5,
                                verbose_eval=1000,
                                feval=crps_eval)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[741]	training's crps: 0.0114998	valid_1's crps: 0.0128315
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[761]	training's crps: 0.0115215	valid_1's crps: 0.0127737
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[726]	training's crps: 0.0115573	valid_1's crps: 0.0127044
Training until validation scores don't improve for 5 rounds


KeyboardInterrupt: 

In [ ]:
0.012911,0.0128706,0.012782,0.0127382,0.0131847

In [357]:
#[0.0128264, 0.0127599, 0.0126589, 0.0126459,0.0130807]
#drop new y, orientation_std(maybe),Dir(maybe),back_oriented_down_field(maybe),Vertical Speed(maybe),S(maybe),Dis(maybe),
#Position(maybe),0(maybe)

(23171,)

In [295]:
#rusher only  0.0129224 0.0128887, 0.0127877, 0.0127507, 0.01318
np.mean([0.0129342,0.0128689,0.0127562,0.0126962,0.0131435])

0.0128798

In [244]:
params = {'max_depth':5, 'lambda_l1': 1, 'lambda_l2': 1,
 'num_leaves': 30, 'feature_fraction': 0.4,
 'subsample': 0.4, 'min_child_samples': 20,
 'learning_rate': 0.05,
 'num_iterations': 600, 'random_state': 42,
 'objective': 'mape'}

for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              valid_sets=[dtrain,dvalid],
                              verbose_eval=10)

[10]	training's mape: 0.0328486	valid_1's mape: 0.0326991
[20]	training's mape: 0.0322112	valid_1's mape: 0.0321865
[30]	training's mape: 0.0316222	valid_1's mape: 0.0316953
[40]	training's mape: 0.0313034	valid_1's mape: 0.0314854
[50]	training's mape: 0.0309792	valid_1's mape: 0.0312592
[60]	training's mape: 0.0307363	valid_1's mape: 0.0311146
[70]	training's mape: 0.0304999	valid_1's mape: 0.0309864
[80]	training's mape: 0.0303047	valid_1's mape: 0.0309035
[90]	training's mape: 0.0301353	valid_1's mape: 0.0308187
[100]	training's mape: 0.0299895	valid_1's mape: 0.0307476
[110]	training's mape: 0.0298492	valid_1's mape: 0.0306947
[120]	training's mape: 0.0297274	valid_1's mape: 0.0306554
[130]	training's mape: 0.0296239	valid_1's mape: 0.0306352
[140]	training's mape: 0.0295154	valid_1's mape: 0.0306009
[150]	training's mape: 0.0294099	valid_1's mape: 0.0305521
[160]	training's mape: 0.0293131	valid_1's mape: 0.0305211
[170]	training's mape: 0.0292312	valid_1's mape: 0.0304916
[180]	

[390]	training's mape: 0.027807	valid_1's mape: 0.0308634
[400]	training's mape: 0.0277738	valid_1's mape: 0.030855
[410]	training's mape: 0.0277367	valid_1's mape: 0.0308502
[420]	training's mape: 0.0277042	valid_1's mape: 0.0308438
[430]	training's mape: 0.0276799	valid_1's mape: 0.0308383
[440]	training's mape: 0.0276571	valid_1's mape: 0.0308373
[450]	training's mape: 0.0276296	valid_1's mape: 0.0308302
[460]	training's mape: 0.0276099	valid_1's mape: 0.030827
[470]	training's mape: 0.0275784	valid_1's mape: 0.0308211
[480]	training's mape: 0.0275427	valid_1's mape: 0.0308151
[490]	training's mape: 0.0275099	valid_1's mape: 0.0308124
[500]	training's mape: 0.0274814	valid_1's mape: 0.0308074
[510]	training's mape: 0.0274509	valid_1's mape: 0.0307977
[520]	training's mape: 0.0274259	valid_1's mape: 0.0307935
[530]	training's mape: 0.0274059	valid_1's mape: 0.0307889
[540]	training's mape: 0.02738	valid_1's mape: 0.0307844
[550]	training's mape: 0.0273561	valid_1's mape: 0.0307808
[5

In [309]:
from sklearn.ensemble import RandomForestRegressor
y = np.zeros((train_y.shape[0], 199))
def crps1(y_true, y_pred):
    y_true = np.clip(np.cumsum(y_true, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0]) 
for idx, target in enumerate(train_y.reset_index(drop=True).Yards):
    y[idx][99 + target] = 1
models = []
kf = KFold(n_splits=5, random_state=42)
score = []
for i, (tdx, vdx) in enumerate(kf.split(X_train, y)):
    print(f'Fold : {i}')
    X_tra, X_val, y_tra, y_val = X_train[tdx], X_train[vdx], y[tdx], y[vdx]
    model = RandomForestRegressor(bootstrap=False, max_features=0.5, min_samples_leaf=15, min_samples_split=7, n_estimators=100, n_jobs=-1, random_state=42)
    model.fit(X_tra, y_tra)
    score_ = crps1(y_val, model.predict(X_val))
    print(score_)
    score.append(score_)
    models.append(model)
print(np.mean(score))

Fold : 0
0.012558720296237821
Fold : 1


KeyboardInterrupt: 

In [302]:
shape=(1, 2)